In [1]:
# Para esconder las API´s keys
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
#·· HACEMOS LAS IMPORTACIONES NECESARIAS ..
# Para hacer las llamadas a las API´s
import requests
# Para poder leer los ficheros csv y hacer DataFrames
import pandas as pd
# Configuración para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_columns', None) 
# Para hacer ficheros json
import json
# Para esconder la API key
import os 
# Para esconder la API key
from dotenv import load_dotenv

In [3]:
# Leer el archivo env, que tiene las claves - pq las tenemos ocultas y lo guardamos en una variable
load_dotenv()
api_key = os.getenv("api_key_lastfm")

In [21]:
# Conversion de csv y llamada a la API
def extraccion_datos (lista_csv):
    datos_artistas = []
    # Convertimos el csv que tenemos de Spotity con la lista solo de artistas
    # Leemos el csv
    df_lista_artistas = pd.read_csv(str(lista_csv))
    # Lo convertimos en lista
    artistas = list(df_lista_artistas['0'])
    # Iteramos por cada artista de la lista Spotify para sacar sus datos en esta API
    for artista in artistas:
        url = f'http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artista}&api_key={api_key}&format=json' 
        # Hacemos la llamada
        datos_artista = requests.get(url)
        # Guardamos los datos en formato json
        datos = datos_artista.json()
        datos_artistas.append(datos)
    return datos_artistas

In [22]:
# Llamamos a la función de extracción de datos
datos_last_fm = extraccion_datos('lista_artistas_spotify.csv')

In [20]:
datos_last_fm['artist']

{'name': 'SadBoyProlific',
 'url': 'https://www.last.fm/music/SadBoyProlific',
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'extralarge'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'mega'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': ''}],
 'streamable': '0',
 'ontour': '0',
 'stats': {'listeners': '223863', 'playcount': '6295863'},
 'similar': {'artist': [{'name': 'Rxseboy',
    'url': 'https://www.last.fm/music/Rxseboy',
    'image': [{'#text

In [26]:
# Limpieza de datos
# Recogemos los datos filtrados por los campos especificados en un archivo


def filtrado_datos(datos): 
    info_artistas = {'Nombre':[],'Biografia':[], 'Listeners':[], 'Playcount':[], 'Artistas similares':[]}
    # Iteramos por cada elemento recogido en data para sacar los resultados que necesitamos
    for dato in datos: 

        # ponemos excepciones a los errores que nos van dando en el filtrado        
        try:
            info_artistas['Nombre'].append(dato['artist']['name'])
        except KeyError:
            continue
        
        info_artistas['Biografia'].append(dato['artist']['bio']['content'])
        info_artistas['Listeners'].append(dato['artist']['stats']['listeners'])
        info_artistas['Playcount'].append(dato['artist']['stats']['playcount'])

        # Los artistas similares son varios: Los recogemos en una lista
        art_similares = []
        for name in dato['artist']['similar']['artist']:
            art_similares.append(name['name'])
        info_artistas['Artistas similares'].append(art_similares)
    
    df_concatenado =pd.DataFrame(info_artistas)
    df_concatenado.to_csv('lastfm_df_concatenado.csv', index=False)
    
    return df_concatenado


In [27]:
# Llamamos a la función que filtra los datos
data = filtrado_datos(datos_last_fm)

In [28]:
data

,Nombre,Biografia,Listeners,Playcount,Artistas similares
0,Tory Lanez,Tory Lanez was born Daystar Peterson on July 2...,1447287,87028513,"[Bryson Tiller, PARTYNEXTDOOR, Drake, A Boogie..."
1,pinkpantheress,"Victoria Beverly Walker, better known by her s...",1973251,144557566,[]
2,Eyedress,"Idris Vicuña (born May 28, 1990), professional...",1611130,49492074,"[Yot Club, TV Girl, paulie leparik, Mac DeMarc..."
3,Jordana,There are at least 2 artists which use this na...,796769,14249901,"[TV Girl, The Simps, Lunar Vacation, VACATIONS..."
4,Tate McRae,"Tate Rosner McRae (born July 1, 2003) is a Can...",1420625,79341249,"[Madison Beer, Nessa Barrett, Gracie Abrams, Z..."
...,...,...,...,...,...
1874,Quando Rondo,"Tyquian Terrel Bowman (born March 23, 1999), b...",149548,5033914,"[YoungBoy Never Broke Again, NoCap, JayDaYoung..."
1875,Mighty Bay,,111855,645822,"[TisaKorean, Lijay, A$AP ANT, DaBaby, CL4PERS]"
1876,Vin Jay,,48326,867091,"[Gawne, EKOH, Chris Webby, Merkules, Bazanji]"
1877,Fat Nick,"Fat Nick (born September 6, 1994) is a Floridi...",190007,3078652,"[Shakewell, Pouya, BOOBIE LOOTAVELI, Pouya & F..."


In [25]:
# Convertimos los resultados en un DataFrame
df_info_artistas = pd.DataFrame(data)

df_info_artistas

,Nombre,Biografia,Listeners,Playcount,Artistas similares
0,Tory Lanez,Tory Lanez was born Daystar Peterson on July 2...,1447287,87028513,"[Bryson Tiller, PARTYNEXTDOOR, Drake, A Boogie..."
1,pinkpantheress,"Victoria Beverly Walker, better known by her s...",1973251,144557566,[]
2,Eyedress,"Idris Vicuña (born May 28, 1990), professional...",1611130,49492074,"[Yot Club, TV Girl, paulie leparik, Mac DeMarc..."
3,Jordana,There are at least 2 artists which use this na...,796769,14249901,"[TV Girl, The Simps, Lunar Vacation, VACATIONS..."
4,Tate McRae,"Tate Rosner McRae (born July 1, 2003) is a Can...",1420625,79341249,"[Madison Beer, Nessa Barrett, Gracie Abrams, Z..."
...,...,...,...,...,...
1874,Quando Rondo,"Tyquian Terrel Bowman (born March 23, 1999), b...",149548,5033914,"[YoungBoy Never Broke Again, NoCap, JayDaYoung..."
1875,Mighty Bay,,111855,645822,"[TisaKorean, Lijay, A$AP ANT, DaBaby, CL4PERS]"
1876,Vin Jay,,48326,867091,"[Gawne, EKOH, Chris Webby, Merkules, Bazanji]"
1877,Fat Nick,"Fat Nick (born September 6, 1994) is a Floridi...",190007,3078652,"[Shakewell, Pouya, BOOBIE LOOTAVELI, Pouya & F..."


In [13]:
# Convertimos el DataFrame en cvs para su posterior carga en SQL
df_info_artistas.to_csv('lastfm_df_concatenado.csv')